# Correctness

## DB Version
1. [ ] Min int on first db creation
2. [ ] Increments with every modification to a crr datum
3. [ ] Restored from disk on db load
4. [ ] Always unique across all rows for this site

## Site id
1. [ ] Initialized to uuid at startup
2. [ ] Persisted
3. [ ] Loaded from disk if exists on disk
4. [ ] Does not ever change after being set, even between restarts

## Schema modification
1. [ ] create table
  a. [ ] if not exists support
  b. [ ] temp table support
  c. [ ] quoted identifiers
  d. [ ] no primary keys table
  e. [ ] compound primary key table
  f. [ ] single primary key
2. [ ] create index
3. [ ] drop index
4. [ ] drop table
5. [ ] alter table

## Inserts of new rows
1. [ ] version cols start at 0
2. [ ] cl starts at 1
3. [ ] db version incremented
4. [ ] clock record written with new db version and current site id for current row
5. [ ] db version is not in use on any other row
6. [ ] cols have the inserted values
7. [ ] update src is 0

## Updates of rows
1. [ ] version cols for changed rows increment by 1
2. [ ] version cols for unchanges rows do not change
3. [ ] db version is incremented
4. [ ] clock record for this row records new db version that is greater than last recorded db version
5. [ ] db version for the row is globally unique
6. [ ] local updates are always taken -- no conflict resolution required
7. [ ] update src is 0

## Deletes of rows
1. [ ] db version is incremented
2. [ ] clock record for this row records new db version that is greater than last recorded db version
3. [ ] db version for the row is globally unique
4. [ ] local deletes are always taken -- no conflict resolution required
5. [ ] if causal length was odd, it is incremented
6. [ ] if causal length was even, it is unchanged
7. [ ] version columns are unchanged
8. [ ] value columns are unchanged
9. [ ] update src is 0

## Inserts of existing rows
1. [ ] if causal length was odd, it is unchanged
2. [ ] if causal length was even, it is incremented
3. [ ] only cols referenced in insert are changed
4. [ ] version cols are incremented for changed cols
5. [ ] version cols are unchanged for unchanged cols
6. [ ] clock record for this row records new db version that is greater than last recorded db version
7. [ ] db version for the row is globally unique
8. [ ] update src is 0

## Reads
1. [ ] deleted rows (even cl) are not returned
2. [ ] undeleted (odd cl) rows are returned
3. [ ] version cols are not returned
4. [ ] cl is not returned
5. [ ] update src is note returned

## Merging remote changes
1. [ ] merges against a row are idempotent
  a. [ ] merging an old row (by vclock) does not change the new row
  b. [ ] merging a row with an identical copy of itself does not change the row
  c. [ ] reapplications of a merge, after the first, does not impact the state of the row
2. [ ] update src is set to 1
3. [ ] only columns with higher versions are taken
4. [ ] if versions match for a column, the greater value is taken
5. [ ] 

## Computing deltas against remote clock

## Concurrency

In [1]:
import sqlite3
import os
import time
import random
import string
import pathlib

db_file = './analyze/correctness.db'
extension = './dist/cfsqlite'
min_db_v = -9223372036854775807

pathlib.Path(db_file).unlink(missing_ok=True)

c = sqlite3.connect(db_file)
c.enable_load_extension(True)
c.load_extension(extension)

# DB Version

In [2]:
# C1
assert c.execute("SELECT cfsql_dbversion()").fetchone()[0] == min_db_v

# C2
c.execute("select cfsql('create table foo (id primary key, a)')")
c.execute("insert into foo values (1, 2)");
assert c.execute("SELECT cfsql_dbversion()").fetchone()[0] == min_db_v + 1
c.execute("update foo set a = 3 where id = 1")
assert c.execute("SELECT cfsql_dbversion()").fetchone()[0] == min_db_v + 2
c.execute("delete from foo where id = 1")
assert c.execute("SELECT cfsql_dbversion()").fetchone()[0] == min_db_v + 3

c.close()

c = sqlite3.connect(db_file)
c.enable_load_extension(True)
c.load_extension(extension)

# C3
print(c.execute("SELECT cfsql_dbversion()").fetchone()[0])
assert c.execute("SELECT cfsql_dbversion()").fetchone()[0] == min_db_v + 3

# C4 -- untested
c.close()

0


AssertionError: 

# Site ID